# Q&A about the document

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/QA_for_document.ipynb.ipynb)

In [3]:
#%%capture
!pip install chromadb
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 7.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.1/910.1 kB 16.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 13.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import OpenAI

In [5]:
file_path = "/home/jovyan/langchain/data/sample.txt"


#  データの用意
with open(file_path) as f:
    dragonball_txt = f.read()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(dragonball_txt)


query = "Kaggle社内Meet upとはどんなものですか?"

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query)

KeyboardInterrupt: 

In [ ]:
llm = OpenAI(temperature=0, model_name="text-davinci-002")

### Stuff
* 全ての関連データをコンテキストとしてプロンプトに詰め込み，言語モデルに渡す手法
    * メリット：LLMへの呼び出しは一回のみになり，テキスト生成時にLLMは一度に全てのデータを参照できる
    * デメリット：LLMのコンテキストの長さによる制限により大きなデータでは機能しない

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff")
#chain.run(input_documents=docs, question=query)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

### Map Reduce
* 関連データをチャンクに分割し，チャンクごとにプロンプトを作り言語モデルに渡す．その後，それらの結果を結合するプロンプトを言語モデルに渡す
    * メリット：Stuffingより大きなデータが扱える．チャンクのLLMの呼び出しを並列実行できる．
    * デメリット：Stuffingより多くの回数のLLMの呼び出しが必要になる．また最後の結合により一部のデータを失う

In [ ]:
chain = load_qa_chain(llm, chain_type="map_reduce")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
chain = load_qa_chain(llm, chain_type="refine")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)